In [2]:
import pandas as pd

pd.set_option('display.max_rows', 100)
pd.set_option('display.max_columns', 30)

# Invoice 1343605689

In [75]:
df_infra2_agg = catalog.load('infra2_cur_dataset_agg')
df_infra2_agg

[07/12/23 21:58:23] INFO     Loading data from 'infra2_cur_dataset_agg' (CSVDataSet)...         ]8;id=380716;file:///home/ubuntu/datarain-aws-cur-finops/venv_kedro/lib/python3.10/site-packages/kedro/io/data_catalog.py\data_catalog.py]8;;\:]8;id=508155;file:///home/ubuntu/datarain-aws-cur-finops/venv_kedro/lib/python3.10/site-packages/kedro/io/data_catalog.py#345\345]8;;\

,bill_invoice_id,bill_billing_entity,bill_invoicing_entity,bill_payer_account_id,payer_account_name,line_item_usage_account_id,usage_account_name,line_item_line_item_type,line_item_usage_start_date,line_item_usage_end_date,product_product_name,line_item_currency_code,line_item_unblended_cost,discount_spp_discount,discount_total_discount
0,1343605689,AWS,Amazon AWS Serviços Brasil Ltda.,981064078967,dataRain Payeer 2,065716851408,SPI - SP Intervention - ENE Medical,Tax,2023-05-01 00:00:00,2023-06-01 00:00:00,AWS Config,USD,0.14,0.0,0.0
1,1343605689,AWS,Amazon AWS Serviços Brasil Ltda.,981064078967,dataRain Payeer 2,065716851408,SPI - SP Intervention - ENE Medical,Tax,2023-05-01 00:00:00,2023-06-01 00:00:00,AWS Data Transfer,USD,0.26,0.0,0.0
2,1343605689,AWS,Amazon AWS Serviços Brasil Ltda.,981064078967,dataRain Payeer 2,065716851408,SPI - SP Intervention - ENE Medical,Tax,2023-05-01 00:00:00,2023-06-01 00:00:00,AWS Key Management Service,USD,0.00,0.0,0.0
3,1343605689,AWS,Amazon AWS Serviços Brasil Ltda.,981064078967,dataRain Payeer 2,065716851408,SPI - SP Intervention - ENE Medical,Tax,2023-05-01 00:00:00,2023-06-01 00:00:00,Amazon Elastic Compute Cloud,USD,101.64,0.0,0.0
4,1343605689,AWS,Amazon AWS Serviços Brasil Ltda.,981064078967,dataRain Payeer 2,065716851408,SPI - SP Intervention - ENE Medical,Tax,2023-05-01 00:00:00,2023-06-01 00:00:00,Amazon Simple Notification Service,USD,0.00,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
183519,1349042489,AWS Marketplace,"Amazon Web Services, Inc.",981064078967,dataRain Payeer 2,901519325564,Estre Ambiental,Usage,2023-05-31 23:00:00,2023-06-01 00:00:00,ZoneMinder: Build Your Own Security Camera So...,USD,0.03,0.0,0.0
183520,1366015353,AWS,Amazon AWS Serviços Brasil Ltda.,981064078967,dataRain Payeer 2,072038246234,SMIT,Refund,2023-05-01,2023-06-01,AWS Support (Business),USD,-57.61,0.0,0.0
183521,1366015353,AWS,Amazon AWS Serviços Brasil Ltda.,981064078967,dataRain Payeer 2,072038246234,SMIT,Tax,2023-05-01,2023-06-01,AWS Support (Business),USD,-7.97,0.0,0.0
183522,1366015353,AWS,Amazon AWS Serviços Brasil Ltda.,981064078967,dataRain Payeer 2,562293755332,Cotrijal,Refund,2023-05-01,2023-06-01,AWS Support (Business),USD,-7.00,0.0,0.0


## Totals

In [77]:
df_consume = df_infra2_agg[df_infra2_agg['bill_invoice_id']==1343605689]
df_consume_usage = df_consume[df_consume['line_item_line_item_type'].isin(['Usage', 'Fee'])]
print(f"Consume charges total: {df_consume_usage['line_item_unblended_cost'].sum()}")

Consume charges total: 32954.20047819261


In [78]:
df_consume['line_item_line_item_type'].unique()

array(['Tax', 'Usage', 'Credit', 'Fee'], dtype=object)

In [79]:
df_consume = df_infra2_agg[df_infra2_agg['bill_invoice_id']==1343605689]
df_consume_tax = df_consume[df_consume['line_item_line_item_type']=='Tax']
print(f"Consume tax total: {df_consume_tax['line_item_unblended_cost'].sum()}")

Consume tax total: 3610.5000000000005


In [80]:
spp_discount = df_infra2_agg[df_infra2_agg['bill_invoice_id']==1343605689]['discount_spp_discount'].sum()
print(f'SPP Discount: {spp_discount}')

SPP Discount: -4007.668503886399


In [81]:
df_consume = df_infra2_agg[df_infra2_agg['bill_invoice_id']==1343605689]
credits = df_consume[df_consume['line_item_line_item_type']=='Credit']['line_item_unblended_cost'].sum()
print(f'Consume discount total: {credits}')

Consume discount total: -2843.3143923311


## Subtotals allocated for each account

In [83]:
df_infra2_agg_totals = df_infra2_agg.groupby(by=['bill_invoice_id', 'bill_payer_account_id', 'line_item_usage_account_id', 
                                                 'usage_account_name', 'product_product_name', 'line_item_line_item_type']).agg(
    {'line_item_unblended_cost': 'sum', 'discount_spp_discount': 'sum'}).reset_index()
df_infra2_agg_totals

,bill_invoice_id,bill_payer_account_id,line_item_usage_account_id,usage_account_name,product_product_name,line_item_line_item_type,line_item_unblended_cost,discount_spp_discount
0,1343605689,981064078967,065716851408,SPI - SP Intervention - ENE Medical,AWS Config,Tax,0.140000,0.000000
1,1343605689,981064078967,065716851408,SPI - SP Intervention - ENE Medical,AWS Config,Usage,1.146000,-0.126060
2,1343605689,981064078967,065716851408,SPI - SP Intervention - ENE Medical,AWS Data Transfer,Tax,0.260000,0.000000
3,1343605689,981064078967,065716851408,SPI - SP Intervention - ENE Medical,AWS Key Management Service,Tax,0.000000,0.000000
4,1343605689,981064078967,065716851408,SPI - SP Intervention - ENE Medical,AWS Key Management Service,Usage,0.000504,-0.000055
...,...,...,...,...,...,...,...,...
783,1349042489,981064078967,901519325564,Estre Ambiental,ZoneMinder: Build Your Own Security Camera So...,Usage,4.470000,0.000000
784,1366015353,981064078967,072038246234,SMIT,AWS Support (Business),Refund,-57.610000,0.000000
785,1366015353,981064078967,072038246234,SMIT,AWS Support (Business),Tax,-7.970000,0.000000
786,1366015353,981064078967,562293755332,Cotrijal,AWS Support (Business),Refund,-7.000000,0.000000


In [62]:
df_infra2_agg_pivoted = df_infra2_agg_totals.pivot(index=['bill_invoice_id', 'bill_payer_account_id', 
                                  'line_item_usage_account_id', 'usage_account_name', 
                                  'product_product_name', 'discount_spp_discount'],
                              columns='line_item_line_item_type', values='line_item_unblended_cost').reset_index()
df_infra2_agg_pivoted = df_infra2_agg_pivoted.fillna(0)

df_infra2_agg_pivoted = df_infra2_agg_pivoted.groupby(by=['bill_invoice_id', 'bill_payer_account_id', 
                                  'line_item_usage_account_id', 'usage_account_name', 
                                  'product_product_name']).agg({'discount_spp_discount': 'sum', 
                                                                    'Credit': 'sum',
                                                                    'Fee': 'sum',
                                                                    'Refund': 'sum',
                                                                    'Tax': 'sum',
                                                                    'Usage': 'sum'}).reset_index()

df_result = df_infra2_agg_pivoted[['bill_invoice_id', 'bill_payer_account_id', 
                                  'line_item_usage_account_id', 'usage_account_name', 
                                  'product_product_name']].copy()
df_result['charges'] = df_infra2_agg_pivoted['Usage'] + df_infra2_agg_pivoted['Fee']
df_result['discount_spp'] = df_infra2_agg_pivoted['discount_spp_discount']
df_result['credits'] = df_infra2_agg_pivoted['Credit']
df_result['tax'] = df_infra2_agg_pivoted['Tax']
df_result['total'] = df_result['charges'] + df_result['discount_spp'] + df_result['credits'] + df_result['tax']
df_result

line_item_line_item_type,bill_invoice_id,bill_payer_account_id,line_item_usage_account_id,usage_account_name,product_product_name,charges,discount_spp,credits,tax,total
0,1343605689,981064078967,065716851408,SPI - SP Intervention - ENE Medical,AWS Config,1.146000,-1.260600e-01,0.0,0.14,1.159940
1,1343605689,981064078967,065716851408,SPI - SP Intervention - ENE Medical,AWS Data Transfer,0.000000,0.000000e+00,0.0,0.26,0.260000
2,1343605689,981064078967,065716851408,SPI - SP Intervention - ENE Medical,AWS Key Management Service,0.000504,-5.544000e-05,0.0,0.00,0.000449
3,1343605689,981064078967,065716851408,SPI - SP Intervention - ENE Medical,Amazon Elastic Compute Cloud,826.951723,-9.096469e+01,0.0,101.64,837.627034
4,1343605689,981064078967,065716851408,SPI - SP Intervention - ENE Medical,Amazon Simple Notification Service,0.000002,-2.200000e-07,0.0,0.00,0.000002
...,...,...,...,...,...,...,...,...,...,...
386,1349042489,981064078967,563324065975,Symbiomics,OpenVPN Access Server (5 Connected Devices),71.424000,0.000000e+00,0.0,0.00,71.424000
387,1349042489,981064078967,901519325564,Estre Ambiental,Reverse Proxy Server using NGINX on Ubuntu,0.144000,0.000000e+00,0.0,0.00,0.144000
388,1349042489,981064078967,901519325564,Estre Ambiental,ZoneMinder: Build Your Own Security Camera So...,4.470000,0.000000e+00,0.0,0.00,4.470000
389,1366015353,981064078967,072038246234,SMIT,AWS Support (Business),0.000000,0.000000e+00,0.0,-7.97,-7.970000


In [65]:
df_result[df_result['bill_invoice_id']==1343605689]['charges'].sum()

32954.2004781926

In [66]:
df_result[df_result['bill_invoice_id']==1343605689]['discount_spp'].sum()

-4007.668503886399

In [67]:
df_result[df_result['bill_invoice_id']==1343605689]['credits'].sum()

-2843.3143923311004

In [68]:
df_result[df_result['bill_invoice_id']==1343605689]['tax'].sum()

3610.5

In [69]:
df_result[df_result['bill_invoice_id']==1343605689]['total'].sum()

29713.7175819751

In [55]:
df_infra2_agg_pivoted[df_infra2_agg_pivoted['bill_invoice_id']==1343605689]['Usage'].sum() + \
df_infra2_agg_pivoted[df_infra2_agg_pivoted['bill_invoice_id']==1343605689]['Fee'].sum()

32954.2004781926

In [56]:
df_infra2_agg_pivoted[df_infra2_agg_pivoted['bill_invoice_id']==1343605689]['discount_spp_discount'].sum()

-4007.668503886399

In [57]:
df_infra2_agg_pivoted[df_ainfra2_agg_pivoted['bill_invoice_id']==1343605689]['Credit'].sum()

-2843.3143923311004

In [60]:
df_infra2_agg_pivoted[df_infra2_agg_pivoted['bill_invoice_id']==1343605689]['Tax'].sum()

3610.5

In [51]:
(df_infra2_agg_pivoted['Usage'] + df_infra2_agg_pivoted['Fee']).sum()

33556.2324781926

In [46]:
df_infra2_agg_pivoted['discount_spp_discount'].astype(float)

0     -1.260600e-01
1      0.000000e+00
2     -5.544000e-05
3     -9.096469e+01
4     -2.200000e-07
           ...     
386    0.000000e+00
387    0.000000e+00
388    0.000000e+00
389    0.000000e+00
390    0.000000e+00
Name: discount_spp_discount, Length: 391, dtype: float64

In [74]:
df_agg = catalog.load('infra2_cur_dataset_agg')
df_agg_pivoted = df_agg.pivot(index=['bill_invoice_id', 'bill_payer_account_id', 
                                            'line_item_usage_account_id', 'usage_account_name', 
                                            'product_product_name', 'discount_spp_discount'],
                                            columns='line_item_line_item_type', 
                                            values='line_item_unblended_cost').reset_index()

df_agg_pivoted

[07/12/23 21:58:12] INFO     Loading data from 'infra2_cur_dataset_agg' (CSVDataSet)...         ]8;id=831203;file:///home/ubuntu/datarain-aws-cur-finops/venv_kedro/lib/python3.10/site-packages/kedro/io/data_catalog.py\data_catalog.py]8;;\:]8;id=11710;file:///home/ubuntu/datarain-aws-cur-finops/venv_kedro/lib/python3.10/site-packages/kedro/io/data_catalog.py#345\345]8;;\

╭─────────────────────────────── Traceback (most recent call last) ────────────────────────────────╮
│ in <module>:2                                                                                    │
│                                                                                                  │
│   1 df_agg = catalog.load('infra2_cur_dataset_agg')                                              │
│ ❱ 2 df_agg_pivoted = df_agg.pivot(index=['bill_invoice_id', 'bill_payer_account_id',             │
│   3 │   │   │   │   │   │   │   │   │   │   │   'line_item_usage_account_id', 'usage_account     │
│   4 │   │   │   │   │   │   │   │   │   │   │   'product_product_name', 'discount_spp_discou     │
│   5 │   │   │   │   │   │   │   │   │   │   │   columns='line_item_line_item_type',              │
│                                                                                                  │
│ /home/ubuntu/datarain-aws-cur-finops/venv_kedro/lib/python3.10/site-packages/pandas/core/frame.p │
│ y:8414 in pivot                                                                                  │
│                                                                                                  │
│    8411 │   def pivot(self, *, columns, index=lib.NoDefault, values=lib.NoDefault) -> DataFrame: │
│    8412 │   │   from pandas.core.reshape.pivot import pivot                                      │
│    8413 │   │                                                                                    │
│ ❱  8414 │   │   return pivot(self, index=index, columns=columns, values=values)                  │
│    8415 │                                                                                        │
│    8416 │   _shared_docs[                                                                        │
│    8417 │   │   "pivot_table"                                                                    │
│                                                                                                  │
│ /home/ubuntu/datarain-aws-cur-finops/venv_kedro/lib/python3.10/site-packages/pandas/core/reshape │
│ /pivot.py:557 in pivot                                                                           │
│                                                                                                  │
│   554 │   # error: Argument 1 to "unstack" of "DataFrame" has incompatible type "Union           │
│   555 │   # [List[Any], ExtensionArray, ndarray[Any, Any], Index, Series]"; expected             │
│   556 │   # "Hashable"                                                                           │
│ ❱ 557 │   result = indexed.unstack(columns_listlike)  # type: ignore[arg-type]                   │
│   558 │   result.index.names = [                                                                 │
│   559 │   │   name if name is not lib.NoDefault else None for name in result.index.names         │
│   560 │   ]                                                                                      │
│                                                                                                  │
│ /home/ubuntu/datarain-aws-cur-finops/venv_kedro/lib/python3.10/site-packages/pandas/core/series. │
│ py:4313 in unstack                                                                               │
│                                                                                                  │
│   4310 │   │   """                                                                               │
│   4311 │   │   from pandas.core.reshape.reshape import unstack                                   │
│   4312 │   │                                                                                     │
│ ❱ 4313 │   │   return unstack(self, level, fill_value)                                           │
│   4314 │                                                                                         │
│   4315 │   # ----------------------------------------------

In [84]:
merged = catalog.load('infra2_cur_dataset_merged')

[07/12/23 22:06:19] INFO     Loading data from 'infra2_cur_dataset_merged' (CSVDataSet)...      ]8;id=18115;file:///home/ubuntu/datarain-aws-cur-finops/venv_kedro/lib/python3.10/site-packages/kedro/io/data_catalog.py\data_catalog.py]8;;\:]8;id=22134;file:///home/ubuntu/datarain-aws-cur-finops/venv_kedro/lib/python3.10/site-packages/kedro/io/data_catalog.py#345\345]8;;\

[07/12/23 22:06:55] WARNING  /home/ubuntu/datarain-aws-cur-finops/venv_kedro/lib/python3.10/site-pa ]8;id=542529;file:///usr/lib/python3.10/warnings.py\warnings.py]8;;\:]8;id=625002;file:///usr/lib/python3.10/warnings.py#109\109]8;;\
                             ckages/kedro_datasets/pandas/csv_dataset.py:162: DtypeWarning: Columns                
                             (4,14,17,18,19) have mixed types. Specify dtype option on import or                   
                             set low_memory=False.                                                                 
                               return pd.read_csv(load_path, **self._load_args)                                    
                                                                                                                   

In [86]:
merged['bill_billing_period_start_date'].unique()

array(['2023-05-01'], dtype=object)

In [87]:
merged['bill_billing_period_end_date'].unique()

array(['2023-06-01'], dtype=object)

In [3]:
catalog.list()

['s3_pocs_cur_dataset',
 's3_intelbras_cur_dataset',
 's3_infra2_cur_dataset',
 'accounts_per_organization',
 'pocs_cur_dataset',
 'intelbras_cur_dataset',
 'infra2_cur_dataset',
 'processed_accounts_per_organization',
 'pocs_cur_dataset_merged',
 'intelbras_cur_dataset_merged',
 'infra2_cur_dataset_merged',
 'infra2_cur_dataset_agg',
 'infra2_agg_invoices',
 'parameters',
 'params:data_prep_raw_params',
 'params:data_prep_raw_params.kept_columns']

In [6]:
invoices = catalog.load('infra2_agg_invoices')
invoices

[07/12/23 22:46:55] INFO     Loading data from 'infra2_agg_invoices' (CSVDataSet)...            ]8;id=164351;file:///home/ubuntu/datarain-aws-cur-finops/venv_kedro/lib/python3.10/site-packages/kedro/io/data_catalog.py\data_catalog.py]8;;\:]8;id=585158;file:///home/ubuntu/datarain-aws-cur-finops/venv_kedro/lib/python3.10/site-packages/kedro/io/data_catalog.py#345\345]8;;\

,bill_invoice_id,bill_payer_account_id,bill_billing_period_end_date,bill_billing_period_start_date,line_item_usage_account_id,usage_account_name,product_product_name,charges,discount_spp,credits,tax,total
0,1343605689,981064078967,2023-06-01,2023-05-01,065716851408,SPI - SP Intervention - ENE Medical,AWS Config,1.146000,-1.260600e-01,0.0,0.14,1.159940
1,1343605689,981064078967,2023-06-01,2023-05-01,065716851408,SPI - SP Intervention - ENE Medical,AWS Data Transfer,0.000000,0.000000e+00,0.0,0.26,0.260000
2,1343605689,981064078967,2023-06-01,2023-05-01,065716851408,SPI - SP Intervention - ENE Medical,AWS Key Management Service,0.000504,-5.544000e-05,0.0,0.00,0.000449
3,1343605689,981064078967,2023-06-01,2023-05-01,065716851408,SPI - SP Intervention - ENE Medical,Amazon Elastic Compute Cloud,826.951723,-9.096469e+01,0.0,101.64,837.627034
4,1343605689,981064078967,2023-06-01,2023-05-01,065716851408,SPI - SP Intervention - ENE Medical,Amazon Simple Notification Service,0.000002,-2.200000e-07,0.0,0.00,0.000002
...,...,...,...,...,...,...,...,...,...,...,...,...
386,1349042489,981064078967,2023-06-01,2023-05-01,563324065975,Symbiomics,OpenVPN Access Server (5 Connected Devices),71.424000,0.000000e+00,0.0,0.00,71.424000
387,1349042489,981064078967,2023-06-01,2023-05-01,901519325564,Estre Ambiental,Reverse Proxy Server using NGINX on Ubuntu,0.144000,0.000000e+00,0.0,0.00,0.144000
388,1349042489,981064078967,2023-06-01,2023-05-01,901519325564,Estre Ambiental,ZoneMinder: Build Your Own Security Camera So...,4.470000,0.000000e+00,0.0,0.00,4.470000
389,1366015353,981064078967,2023-06-01,2023-05-01,072038246234,SMIT,AWS Support (Business),0.000000,0.000000e+00,0.0,-7.97,-7.970000


In [7]:
invoices.groupby(by='bill_invoice_id').agg({
    'charges': 'sum',
    'discount_spp': 'sum',
    'credits': 'sum',
    'tax': 'sum',
    'total': 'sum'
})

,charges,discount_spp,credits,tax,total
bill_invoice_id,,,,,
1343605689,32954.200478,-4007.668504,-2843.314392,3610.50,29713.717582
1349042489,602.032000,0.000000,0.000000,0.00,602.032000
1366015353,0.000000,0.000000,0.000000,-8.93,-8.930000
